# TIME AWARE POI RECOMMENDATION SYSTEM 
    Project for CS6670
    
    
BY

*   CS16BTECH11039
*   CS16BTECH11029
*   CS16BTECH11034
*   CS16BTECH11002



# For GOOGLE COLAB

In [1]:
! pip3 install -U -q PyDrive

    100% |████████████████████████████████| 993kB 16.8MB/s 


In [0]:
! pip3 install h5py

In [0]:
import numpy as np
import sklearn as sk
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files
from scipy import sparse
from math import sqrt
import math
from scipy.sparse import *

#### click link and then sign in using your google email

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

### For downloading and extracting the data set see in the left pane of window for the dataset

In [0]:
download = drive.CreateFile({'id': '1_p8KruCi1zMOXl7phAt6FY0L16b8kpde'})
download.GetContentFile('poidata.tar')

In [0]:
download = drive.CreateFile({'id': '1X3gQmWBc4vzf5udsPrqc0XISFZROmK2W'})
download.GetContentFile('simi.csv')

In [0]:
download = drive.CreateFile({'id': '1y2MaxDjIplMtRaWy-7dG9m47VU2ILiRx'})
download.GetContentFile('wuv.csv')

In [0]:
download = drive.CreateFile({'id':'1HcdMaID94WZSKslBKLbZ5clwWe3GYD9b'})
download.GetContentFile('hi.tar.gz')

In [0]:
download = drive.CreateFile({'id':'1HshGNzhNOZEXfDN0QBSKSyMOQCb-KXYZ'})
download.GetContentFile('cult.tar.gz')

In [0]:
! pwd

In [0]:
! tar -xvzf cult.tar.gz

In [0]:

download = drive.CreateFile({'id':'11W8EVfbBtMW8TajayNKqf8JTa4_s2pOg'})
download.GetContentFile('hi')

In [0]:
! tar -xvzf hi.tar.gz

In [0]:
! tar -xvf poidata.tar
! echo $?

# Pre Process

In [0]:
class preProcessPOI:
    
    def __init__(self,fi1,fi2,fi3):
        self.train = open(fi1,"r")
        self.tune = open(fi2,"r")
        self.test = open(fi3,"r")
        count = 0  
        user = []
        loc = []
        for line in self.train:
            l = line.split('\t')    
            user.append(l[0].strip())
            loc.append(l[1].strip())
            count+=1
        self.numUsers = len(set(user))
        self.numLoc = len(set(loc))
        self.numTimeSlots = 24
        self.K = 0
        self.alpha = 0
        self.beta = 0
        self.A = 0
        
        self.data = np.zeros((2321,5596,24))
        
        self.locIdToCoord = np.zeros((self.numLoc,2),dtype = np.float)
        self.timeSimi = np.zeros((self.numTimeSlots,self.numTimeSlots),dtype = np.float)
        self.tot_check_ins = np.zeros((self.numLoc,),dtype = np.int32)
        self.tot_check_ins_time = np.zeros((self.numLoc,self.numTimeSlots),dtype = np.int32)
        self.dateDict = {}
        
    def readFile(self):
        self.train.seek(0)
        dateDic = {}
        lineNo = 0
        for line in self.train:
            line = line.strip()
            arr = line.split('\t')
            locId =int (arr[1].split('_')[1])
            coord  = (arr[2].split(','))
            userId = int (arr[0].split('_')[1])
            timeSlot = int (arr[3].split(':')[0]) 
                            
            noMins = arr[3]
            dateId = int(arr[4])
            

            if dateId in dateDic :
                if userId in dateDic[dateId] :
                    if noMins in dateDic[dateId][userId] :
                        dateDic[dateId][userId][noMins].append(locId )
                    else:
                        dateDic[dateId][userId][noMins] = [locId]     
                else :
                    dateDic[dateId][userId] = {}
                    dateDic[dateId][userId][noMins] = [locId]
            else :
                dateDic[dateId] = {}
                dateDic[dateId][userId] = {}
                dateDic[dateId][userId][noMins] = [locId]
                
            
            lineNo+=1
            self.locIdToCoord[locId,0] = float (coord[0])
            self.locIdToCoord[locId,1] = float (coord[1])
            self.data[userId,locId,timeSlot] = 1
            
            self.tot_check_ins[locId]+=1
            self.tot_check_ins_time[locId,timeSlot] +=1
        
        dateDic2 = {}
        for x in dateDic :
            print(x)
            dateDic2[x] = {}
            
            temp = {}
            tempDic = dateDic[x]
            print(dateDic[x])           
            print(tempDic)
            for user in tempDic :
                tempDic2 = tempDic[user]
                tempsortedKeys = list(tempDic2)
                sortedKeys = sorted(tempsortedKeys)
                sortedData = []
                for y in sortedKeys :
                    sortedData.extend(tempDic2[y])
                temp[user] = sortedData
            print(temp)
            dateDic2[x] = temp

        print(dateDic2)
        self.dateDict = dateDic2
    def getCoord(self,loc):
        return self.locIdToCoord[loc][0],self.locIdToCoord[loc][1]
            
    def createCheckInVector(self,userID,timeSlot):
            temp = np.zeros((self.numLoc,), dtype = np.float)         
            temp[self.data[userID,:,timeSlot] == 1] = 1.0
            return temp

### FOR PRE PROCESSING THE DATA

In [39]:
poiData = preProcessPOI("/content/home/ujjieve/Desktop/poiRec/poidata/Foursquare/train.txt","/content/home/ujjieve/Desktop/poiRec/poidata/Foursquare/tune.txt","/content/home/ujjieve/Desktop/poiRec/poidata/Foursquare/test.txt")
poiData.readFile()
print (poiData.numLoc,poiData.numUsers)

194108
5596 2321


# Temporal Influence

## For computing the time similarity

In [0]:
### For getting rho_t_t' given in formula of section 3.3 of pdf 

def compute_time_simi(poi):
   
    for i in range(poiData.numTimeSlots):
        for j in range(i, poiData.numTimeSlots):
            if i==j:
                poiData.timeSimi[i,j] = 1.0
                continue
            sum = 0.0
            for user in range(poiData.numUsers):
                ci = poiData.createCheckInVector(user, i)
                cj = poiData.createCheckInVector(user, j)
                sum += cosine_similarity(ci.reshape(1,-1),cj.reshape(1,-1))[0]
            poiData.timeSimi[i,j] = poiData.timeSimi[j,i] = sum / poiData.numUsers
        print(i)
         

In [0]:
## TO compute the rho
%time compute_time_simi(poiData) 


In [0]:
np.savetxt('simi.csv',poiData.timeSimi,delimiter = ',')

In [0]:
print(poiData.timeSimi)

In [0]:
timeSimi = np.genfromtxt('simi.csv',delimiter = ',')

In [0]:
print(timeSimi)

In [0]:
wuv_matrix = np.genfromtxt('wuv.csv',delimiter = ',')

In [0]:
print(wuv_matrix)

### For getting the UTP cube matrix with time smoothing enhancement

In [0]:
## for getting the matrix cutl  given in section 3.3 of pdf 
def get_mcutl(poi,timeSimi):
    mcutl = np.zeros((poi.numUsers,poi.numLoc,poi.numTimeSlots), dtype = np.float)
    x = np.sum(timeSimi,axis = 1)
    
    for u in range(poi.numUsers):
        hi = np.genfromtxt('cutl' + str(u) + '.csv',delimiter = ',')
        mcutl[u] =  ( np.dot( hi ,timeSimi )) / x
        
    
    return mcutl   
        


In [0]:
%time mcutl = get_mcutl(poiData,timeSimi)

In [0]:
print(mcutl)

In [0]:
with open('mcutl.txt', 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(mcutl.shape))

    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in mcutl:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%s')

        # Writing out a break to indicate different slices...
        outfile.write('# New slice\n')

### For getting the user - user similarity matrix with time smoothing 

#### To ge the user- user similarity given two users

In [0]:
### for gettin the matrix wuv given in section 3.3 of pdf
def get_wuv(poi,mcutl,u,v):
    z = (mcutl[u] * mcutl[v]).sum(axis = 1).sum()
    x =  np.square(mcutl[u]).sum(axis = 1).sum(axis = 0)
    y =  np.square(mcutl[v]).sum(axis = 1).sum(axis = 0)
    return z/(math.sqrt(x) * math.sqrt(y))


In [0]:
%time x= get_wuv(poiData,mcutl,0,12)
print(x)

CPU times: user 2.86 ms, sys: 24 µs, total: 2.89 ms
Wall time: 2.83 ms
0.01168226482043208


#### To get the complete w_uv matrix

In [0]:
def get_wuv_matrix(poi,mcutl):
    wuv = np.zeros(shape = (poi.numUsers,poi.numUsers), dtype = np.float)
    for i in range(poi.numUsers):
        for j in range(i+1,poi.numUsers):
            wuv[i,j] = get_wuv(poi,mcutl,i,j)
        #print(i)
    return wuv

In [0]:
%time wuv_matrix = get_wuv_matrix(poiData,mcutl)

CPU times: user 50min, sys: 1min 17s, total: 51min 18s
Wall time: 51min 20s


In [0]:
for i in range(poiData.numUsers):
    wuv_matrix[i,i] = 1.
print(wuv_matrix)

[[1.         0.         0.         ... 0.00367704 0.         0.        ]
 [0.         1.         0.0060652  ... 0.0690119  0.00707514 0.01820557]
 [0.         0.         1.         ... 0.         0.         0.00719697]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


## For getting the temporal influence from the train data

In [0]:
## for getting the final temporal influence given in section 3.3 
def get_temporal_inf(poi,mcutl,wuv_matrix,timeSimi):
    h = np.matmul(mcutl,timeSimi)
    prediction  = np.zeros(shape = (poi.numUsers,poi.numLoc,poi.numTimeSlots), dtype = np.float)
    for v in range(poi.numUsers):
        prediction[v] = np.sum( h * (wuv_matrix[v].reshape((len(wuv_matrix[v]),1,1))) , axis = 0)  
    return prediction


In [0]:

%time z = get_temporal_inf(poiData,mcutl,wuv_matrix,timeSimi)
print(z)

In [0]:
! mkdir ujjieve


In [0]:
for u in range(poiData.numUsers):
    np.savetxt('/content/ujjieve/pred'+str(u)+'.csv',z[u],delimiter = ',')
 

# Spatial Influnce

#### For getting the distance between the two earth coordinates

In [0]:
## Used for getting spatial influence sectino 4.1
def get_distance(coord1,coord2):
    
    lon1,lat1=coord1
    lon2,lat2=coord2

    R=6371000                               # radius of Earth in meters
    phi_1=math.radians(lat1)
    phi_2=math.radians(lat2)

    delta_phi=math.radians(lat2-lat1)
    delta_lambda=math.radians(lon2-lon1)

    a=math.sin(delta_phi/2.0)**2+\
       math.cos(phi_1)*math.cos(phi_2)*\
       math.sin(delta_lambda/2.0)**2
    c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
    return R*c / 1000.0


In [0]:
print ( get_distance( poiData.getCoord(0),poiData.getCoord(1) ) )

7.898104656555957


#### For getting the distances between every pair of location

In [0]:
def getAllDistances(poi):
  nL = poiData.numLoc
#   nL = 2
  x = np.array([get_distance(poi.locIdToCoord[i],poi.locIdToCoord[j]) for i in range(nL) for j in range(i+1,nL)])
  tri = np.zeros((nL, nL))
  tri[np.triu_indices(nL, 1)] = x
  return tri + tri.T

### For getting the willingness from the train data

In [0]:
def getWillingness(poi):
    res =  {}
    temp = poi.dateDict
    for x in temp :
        tempx = temp[x]
        for u in tempx :
            tempxu = tempx[u]
            for loc1Ind in  range(len(tempxu) - 1 ) :
                loc2Ind = loc1Ind + 1
                loc1 = tempxu[loc1Ind]
                loc2 = tempxu[loc2Ind]
                if loc1 == loc2 :
                    continue
                coord1 = (poi.locIdToCoord[loc1][0],poi.locIdToCoord[loc1][1])
                coord2 = (poi.locIdToCoord[loc2][0],poi.locIdToCoord[loc2][1])
                distance =  get_distance(coord1,coord2)
                if distance > int(distance) + 0.5:
                    key = int(distance) + 1
                else:
                    key = int(distance) + 0.5
                
                if key in res:
                    res[key] +=1
                else:
                    res[key] = 2
    return res

In [7]:
willing= getWillingness(poiData)
print(willing)

{5: 994, 0.5: 24024, 10.5: 546, 1: 4894, 4.5: 1220, 8: 614, 7: 776, 1.5: 3263, 10: 550, 13: 330, 3: 1750, 11: 665, 3.5: 1329, 12: 438, 7.5: 641, 2.5: 2156, 6.5: 803, 6: 986, 5.5: 937, 2: 2575, 18: 102, 12.5: 416, 24: 29, 19: 151, 15.5: 197, 14: 208, 8.5: 544, 4: 1175, 9: 503, 15: 208, 13.5: 293, 24.5: 23, 19.5: 90, 17: 174, 16: 173, 9.5: 533, 11.5: 570, 18.5: 99, 28.5: 12, 23: 49, 16.5: 210, 14.5: 234, 17.5: 174, 27: 21, 21: 46, 22.5: 91, 20: 50, 21.5: 40, 31.5: 7, 20.5: 45, 34.5: 2, 36: 2, 22: 49, 23.5: 28, 27.5: 15, 25.5: 18, 29: 8, 25: 34, 30.5: 9, 34: 3, 26: 14, 26.5: 13, 32.5: 3, 28: 9, 30: 15, 31: 3, 33: 2, 35: 2, 32: 2, 29.5: 7, 33.5: 4}


#### Converting it into log function

In [8]:
distances = list(willing)
print(distances)
print(np.log(distances))
will = [ willing[x] for x in distances]
print (will)
print (np.log(will))
log_will = np.log(will)
log_dist = np.log(distances)

[5, 0.5, 10.5, 1, 4.5, 8, 7, 1.5, 10, 13, 3, 11, 3.5, 12, 7.5, 2.5, 6.5, 6, 5.5, 2, 18, 12.5, 24, 19, 15.5, 14, 8.5, 4, 9, 15, 13.5, 24.5, 19.5, 17, 16, 9.5, 11.5, 18.5, 28.5, 23, 16.5, 14.5, 17.5, 27, 21, 22.5, 20, 21.5, 31.5, 20.5, 34.5, 36, 22, 23.5, 27.5, 25.5, 29, 25, 30.5, 34, 26, 26.5, 32.5, 28, 30, 31, 33, 35, 32, 29.5, 33.5]
[ 1.60943791 -0.69314718  2.35137526  0.          1.5040774   2.07944154
  1.94591015  0.40546511  2.30258509  2.56494936  1.09861229  2.39789527
  1.25276297  2.48490665  2.01490302  0.91629073  1.87180218  1.79175947
  1.70474809  0.69314718  2.89037176  2.52572864  3.17805383  2.94443898
  2.74084002  2.63905733  2.14006616  1.38629436  2.19722458  2.7080502
  2.60268969  3.19867312  2.97041447  2.83321334  2.77258872  2.2512918
  2.44234704  2.91777073  3.34990409  3.13549422  2.80336038  2.67414865
  2.86220088  3.29583687  3.04452244  3.11351531  2.99573227  3.06805294
  3.44998755  3.02042489  3.54095932  3.58351894  3.09104245  3.15700042
  3.31418

#### For Computing A and K parameters using Least squares regression method

In [9]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(log_dist.reshape(-1,1), log_will)
print (reg.coef_)
print(reg.intercept_)
poiData.K = reg.coef_
poiData.A = reg.intercept_

[-2.23069687]
10.40793976121805


#### For getting the willingness given the distance after obtaining the parameters

In [0]:
def willingness(dist,poi):
    ## Tune a and k using
    if dist > int(dist) + 0.5:
        key = int(dist) + 1
    else:
        key = int(dist) + 0.5
    return poi.A*(key**poi.K)

In [0]:
print(poiData.K)

[-2.23069687]


### For getting the probability of moving to a destination given an origin



In [0]:

def prob_loc(loc1,loc2,poi):
    coord1 = poi.getCoord(loc1)
    coord2 = poi.getCoord(loc2)
    wil = willingness(get_distance(coord1,coord2),poi)
    tot_wil = 0.0
    x = [ willingness(get_distance(coord1,poi.getCoord(t)),poi) for t in range(poi.numLoc)]
    x[loc1]  = 0.0
    tot_wil = np.sum(x)
    return wil / tot_wil
    

In [25]:
print ( get_distance(poiData.getCoord(0),poiData.getCoord(1)) )
%time x  = prob_loc(0,1,poiData) 
print(x)

7.898104656555957
CPU times: user 44.2 ms, sys: 1.93 ms, total: 46.2 ms
Wall time: 46.3 ms
[1.87678204e-05]


### For getting the matrix of above 

In [0]:
def getAllProbLoc(poi ):
  allDis = getAllDistances(poiData)
  wi = np.multiply(np.power(np.rint(allDis),poi.K),poi.A)
  x , y = np.where(wi == np.inf)
  wi[x,y] = 0
  sumWi = np.sum(wi,axis=1)
  temp = sumWi - wi.T 
  plL = np.divide( wi ,temp)
  plL[x,y] = 1
  return plL

In [14]:
%time conditionalProb = getAllProbLoc(poiData)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in power
  This is separate from the ipykernel package so we can avoid doing imports until


0  if this is not zero a blunder happend
CPU times: user 1min 3s, sys: 298 ms, total: 1min 3s
Wall time: 1min 4s


### For getting the prior probability using temporal popularity

In [0]:
## sectino 4.2 for getting p_t(l)
def get_prior_prob(loc,poi,time_slot):
    long_term_pop = poi.tot_check_ins[loc] / np.sum(poi.tot_check_ins)
    temporal_pop = poi.tot_check_ins_time[loc,time_slot] / np.sum(poi.tot_check_ins_time[:,time_slot])
    return (poi.beta * temporal_pop)  + ( ( 1- poi.beta ) * long_term_pop )

In [0]:
poiData.beta = 0.7
%time x = get_prior_prob(0,poiData,8)
print (get_prior_prob(0,poiData,8))

CPU times: user 800 µs, sys: 58 µs, total: 858 µs
Wall time: 761 µs
0.0001523857271965644


## For getting the spatial Influence using temporal popularity

In [0]:
## For getting the final spatial influence cutl given in section 4.3
def get_spatial_inf(poi,user,time,loc):
    x = np.zeros((poi.numLoc,))
    for t in range(poi.numTimeSlots):
        y = poi.createCheckInVector(user,t)
        x = np.logical_or(y,x)
    l = [t for t in range(poi.numLoc)]
    l = np.array(l)
    loc_hist = l[x == True]
    post = [prob_loc(loc,t,poi) for t in loc_hist]

    return get_prior_prob(loc,poi,time) * np.prod(post)
    

In [0]:
%time spa = get_spatial_inf(poiData,243,8,22)


CPU times: user 290 ms, sys: 3.84 ms, total: 294 ms
Wall time: 294 ms


In [0]:
poiData.beta = 0.5
%time print (get_prior_prob(0,poiData,8))
def getAllPriorProb(poi):
  return np.array([get_prior_prob(l,poi,t) for l in range(poi.numLoc) for t in range(poi.numTimeSlots)]).reshape((poi.numLoc,poi.numTimeSlots))

In [18]:
%time allPriorProb = getAllPriorProb(poiData)

CPU times: user 6.45 s, sys: 10.5 ms, total: 6.46 s
Wall time: 6.46 s


### For computing the UTP cube incorporating the spatial influence

In [0]:
# poi , condProb(loc x loc) , allPrior(loc x timeSlot)
def getMLEScore(poi , condProb):
  uL = (np.sum(poi.data,axis=2) > 0)*1
  print(uL.shape)
  pUL = np.ones((poi.numUsers,poi.numLoc))
  for i in range(poi.numUsers):
    for j in range(poi.numLoc):
      temp = np.array(uL[i,:] * condProb[j,:])
      temp[temp == 0] = 1
      pUL[i][j] = np.prod(temp)
  return pUL
  

In [20]:
%time sp = getMLEScore(poiData,conditionalProb)

(2321, 5596)
CPU times: user 10min 4s, sys: 229 ms, total: 10min 5s
Wall time: 10min 5s


In [0]:
def getSpatialScore(mle , prior ,timeSlots):
  finalScore = []
  for t in range(timeSlots) :
    finalScore.append(np.multiply(mle,prior[:,t]))
  return np.array(finalScore)

In [22]:
print(allPriorProb.shape)
%time cULTspatial = getSpatialScore(sp,allPriorProb,24)

(5596, 24)
CPU times: user 2.61 s, sys: 1.29 s, total: 3.9 s
Wall time: 3.9 s


In [0]:
nU = 2321 
nL = 5596
nT = 24
prediction  = np.zeros(shape = (nU,nL,nT), dtype = np.float)

In [0]:
for u in range(nU):
    prediction[u] = np.genfromtxt('content/ujjieve/pred'+str(u)+'.csv',delimiter = ',')

In [0]:
print(prediction)

In [0]:
spatial  = np.zeros(shape = (nU,nL,nT), dtype = np.float)
for u in range(nU):
    spatial[u] = np.genfromtxt('content/cult/cULTspatial'+str(u)+'.csv',delimiter = ',')

In [0]:
# for u in range(poiData.numUsers):
#     prediction[u] = np.genfromtxt('content/ujjieve/pred'+str(u)+'.csv',delimiter = ',')

# For getting the combined UTP cube incorporating both temporal and spatial influence

In [0]:
def get_Combined(cULT , cULP , alpha):
    minCULT = np.amin(cULT)
    maxCULT = np.amax(cULT)
    minCULP = np.amin(cULP)
    maxCULP = np.amax(cULP)
    return (alpha * ((cULT-minCULT)/(maxCULT-minCULT)) +  ((1-alpha) * ((cULP-minCULP)/(maxCULP-minCULP))))

In [0]:
total = get_Combined(prediction,spatial,.2)

In [0]:
! mkdir final3

In [0]:
for u in range(nU):
  np.savetxt("/content/final3/final"+str(u)+".csv",total[u],delimiter=",")

In [0]:
! tar -czvf /content/final1.tar.gz /content/final1/

In [0]:
total = np.zeros(shape = (nU,nL,nT),dtype = np.float)
for u in range(nU):
    total[u] = np.genfromtxt('final3/final'+str(u)+'.csv',delimiter = ',')

# For getting Computing the Precision and recall given the test data and the UTP cube

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
def getPrecisionAndRecall(yTrue , yPred ):
    yTrue = yTrue.flatten()
    yPred = yPred.flatten()
    tn, fp, fn, tp = confusion_matrix(yTrue,yPred).ravel()
    precs = tp / (tp + fp) ;
    recal = tp / (tp + fn) ;
    return precs , recal ;

In [0]:
def getCombinedPrecisionRecall(finalScore , groundT , k , poi):
    #k is max allowed predictions for each user
    yP = np.zeros(groundT.shape)
    nU = poi.numUsers
    nT = poi.numTimeSlots
    nL = poi.numLoc
    print(finalScore)
    print(groundT)
    if finalScore.shape != groundT.shape :
        return -1 , -1
    for u in range(nU):
        for t in range(nT):
            temp = finalScore[u,:,t] 
            temp = np.multiply(temp,-1) #makes asd to des
            inds = temp.argsort()[:k] #smallest k numbers
            yP[u,inds,t] = 1

    prec = np.zeros(nT)
    reca = np.zeros(nT)
    for t in range(nT) :
        prec[t] , reca[t] = getPrecisionAndRecall(groundT[:,:,t],yP[:,:,t])

    avgPrec = np.sum(prec) / nT
    avgReca = np.sum(reca) / nT
    return avgPrec , avgReca 

In [14]:
pre,rec = getCombinedPrecisionRecall(total,poiData.data,5,poiData)

prec reca at  0 = 0.0009136592051164915 0.02237136465324385
prec reca at  1 = 0.0001827318410232983 0.006644518272425249
prec reca at  2 = 9.136592051164915e-05 0.004784688995215311
prec reca at  3 = 0.0 0.0
prec reca at  4 = 9.136592051164915e-05 0.007751937984496124
prec reca at  5 = 0.0008222932846048424 0.07627118644067797
prec reca at  6 = 0.0009136592051164915 0.054945054945054944
prec reca at  7 = 0.0014618547281863865 0.04804804804804805
prec reca at  8 = 0.0007309273640931932 0.015444015444015444
prec reca at  9 = 0.0009136592051164915 0.015174506828528073
prec reca at  10 = 0.0015532206486980357 0.02122347066167291
prec reca at  11 = 0.0014618547281863865 0.01593625498007968
prec reca at  12 = 0.0025582457743261763 0.01683704149128082
prec reca at  13 = 0.002832343535861124 0.01732811626607043
prec reca at  14 = 0.003015075376884422 0.02091254752851711
prec reca at  15 = 0.0032891731384193696 0.024489795918367346
prec reca at  16 = 0.003015075376884422 0.022372881355932205
pr

In [16]:
print (rec,pre)

0.020901255274266412 0.0017549870564945941
